In [106]:
import websockets as ws
import json
from datetime import datetime
import time
import math


In [9]:
import asyncio
from websocket_communication import WebSocket

asyncio.run(WebSocket.connect)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [107]:
async def send_boot_notification(websocket):
        
        print("IM BOOT")
        msg = [2, "0jdsEnnyo2kpCP8FLfHlNpbvQXosR5ZNlh8v", "BootNotification", {
            "chargePointVendor": "AVT-Company",
            "chargePointModel": "AVT-Express",
            "chargePointSerialNumber": "avt.001.13.1",
            "chargeBoxSerialNumber": "avt.001.13.1.01",
            "firmwareVersion": "0.9.87",
            "iccid": "",
            "imsi": "",
            "meterType": "AVT NQC-ACDC",
            "meterSerialNumber": "avt.001.13.1.01"}]
        msg_send = json.dumps(msg)
        await websocket.send(msg_send)
        print("SENT")

In [108]:
from datetime import datetime


async def send_data_transfer_request(websocket):
    dt = datetime.now()
    conf_msg = [2,
                    "com.flexicharge",
                    "DataTransfer",
                    {"status": "Accepted"}]
    conf_send = json.dumps(conf_msg)
    print("Sending confirmation: " + conf_send)
    try:
        await websocket.send(conf_send)
        print("Message went away")
    except Exception as e:
        print(str(e))

In [109]:
async def listen_for_response(websocket):
    while True:
        try:
            json_formatted_message = await websocket.recv()
            message = json.loads(json_formatted_message)
            print("Got message:")
            print(message)
            if message[2] == "DataTransfer":
                await send_confirmation(websocket,message)
        except Exception as e:
            print(str(e))
    

In [110]:
try:
    async with ws.connect(
        'ws://18.202.253.30:1337/testnumber13',
        subprotocols=['ocpp1.6'],
        ping_interval=5,
        timeout = None
        ) as ws:
            websocket = ws
            print("success")
            await send_boot_notification(websocket)
            await listen_for_response(websocket)
except Exception as e:
    print("fail")
    print(str(e))

success
IM BOOT
SENT
Got message:
[3, '0jdsEnnyo2kpCP8FLfHlNpbvQXosR5ZNlh8vASDASD', 'BootNotification', {'status': 'Accepted', 'currentTime': 1664265312, 'interval': 86400}]
Got message:
[2, '100009DataTransfer1664265312217', 'DataTransfer', {'vendorId': 'com.flexicharge', 'messageId': 'BootData', 'data': '{"chargerId":100009,"chargingPrice":"7500.00"}'}]
Sending confirmation: [3, "100009DataTransfer1664265312217", "DataTransfer", {"status": "Accepted"}]
Message went away
Got message:
[4, '100009DataTransfer1664265312217', 'GenericError', 'Could not found a previous conversation with this unique id.', {}]


CancelledError: 